In [1]:
# <div class="bCatList2Img">


In [15]:
import json
import os 
import requests
import time
from datetime import datetime,timedelta
from datetime import date
import re
import sys
import urllib, urllib.request, urllib.parse
import random
from scrawl import *

# Date and time
start_time = time.time()
current_time = datetime.now().strftime("%H-%M-%S")
created_on = date.today().strftime("%Y-%m-%d")

# client_id = sys.argv[1]
client_id = '5f69d22ef472d6646f577fa6'  # Europe
site = 'spf_org_publication'
c = Crawl()  # creating obje_
# create directories to store logs.
log_path = c.create_directories(project_path, client_id, site)

# create image directories
image_directory = c.create_image_directories(project_path)
# creating pdf directories
pdf_directory = c.create_pdf_directories(project_path, site)
# logger
logger = log_func(log_path, created_on, current_time)
logger.info("Process Started ...\n")

# initialize variables
skipped_due_to_headline = 0
skipped_due_to_content = 0
skipped_due_to_date = 0
missing_overall_tonality = 0
no_of_data = 0
duplicate_data = 0  
unable_to_fetch_article_url = 0
unable_to_download_pdf = 0
publish_source = 'spf.org'
country = 'Japan'
language = 'English'
images_path = []

home_page = c.download_page('https://www.spf.org/en/reports/')
for i in home_page.split('<div class="bCatList2Img">')[1:]:
    # source_link
    source_link = c.scrap('a\s*href="(.*?)"', i)   

    # handle duplicates
    source_link_query = {'source_link':source_link}
    dic = cl_data.find_one(source_link_query,{'source_link': 1}) 
    if dic:
        duplicate_data += 1
        continue

    time.sleep(random.randint(1,3))

    logger.info(f'Fetching {source_link}\n')

    page = c.download_page(source_link)   
    if page.startswith('Unable to fetch'):
        logger.info(page)
        unable_to_fetch_article_url += 1
        continue    

    source_headline = c.scrap('<title>(.*?)\|', page)

    # skip if headline not found
    if not source_headline:
        logger.info(f'Skipping due to headline {source_link}\n')
        skipped_due_to_headline += 1
        continue

     # Date and time
    pub_date, publish_time = created_on, current_time

    # skip null date
    if not pub_date:
        logger.info(f'Skipping due to date {source_link}\n')            
        skipped_due_to_date += 1
        continue

    # break if date is not today's date
    if pub_date != created_on:
        break    

    # source_content          
    source_content = c.scrap('<div\s*class="bTxtImgLeft">(.*?)</div>\s*</div>\s*</div>\s*</div>',page)
    if not source_content:
        source_content = c.scrap('<div\s*class="bImgTxtRight">(.*?)</div>\s*</div>\s*</div>\s*</div>' , page)
    if not source_content:
        source_content = c.scrap('<div\s*class="contentsInner">(.*?)</h4>' , page) 
    if not source_content:
        source_content = c.scrap('<div\s*class="inner"\s*style="">(.*?)<div\s*class="bPdfLeft">', page)     
    source_content = re.sub('<span\s*style=(.*?)</span>','',source_content,re.S)
    source_content = re.sub('&.*?;','',source_content,re.S)
    source_content = c.strip_html(source_content)
    if not source_content:
        logger.info(f'Skipping due to content {source_link}\n')            
        skipped_due_to_content += 1
        continue

    journalist =c.scrap("'author-name':'(.*?)'",page)

    if not journalist: journalist = 'NA'

    # current date and time 00
    harvest_time = datetime.now().strftime("%H:%M:%S")

    # temp link
    temp_link = source_link

    # headline and content 
    headline = source_headline
    content = source_content

    # overall_tonality
    overall_tonality = ''

    # word count
    word_count = len((source_headline + ' ' + source_content).split())

    html_content = ''

    # image_urls
    image_urls = []
  
    for i in page.split('href')[1:]:
        pdf_url = c.scrap('"(.*?)"', i)
        if pdf_url.endswith('.pdf'):
            if 'http' not in pdf_url:
                pdf_url = 'https://www.spf.org/en/_opri_media/projects/' + pdf_url        
            pdf_name = c.scrap('.*\/(.*)',pdf_url)
            pdf_path = f'{pdf_directory}/{pdf_name}'        
           # download pdf
            pdf = c.download_pdf(pdf_url, pdf_path)
            if pdf.startswith('Unable to fetch'):
                logger.info(pdf) # writes error message with error code
                unable_to_download_pdf += 1
                continue
    # storing the above data in a dictionary
    clientdata ={
                    "client_master" : client_id, 
                    "articleid":client_id,
                    "medium":'Web' ,
                    "searchkeyword":[],
                    "entityname" : [] ,
                    "process_flage":"1",
                    "na_flage":"0",
                    "na_reason":"",
                    "qc_by":"",
                    "qc_on":"",
                    "location":"",
                    "spokeperson":"",
                    "quota":"",
                    "overall_topics":"",
                    "person":"",
                    "overall_entites":"",
                    "overall_tonality": overall_tonality,
                    "overall_wordcount":word_count,
                    "article_subjectivity":"",
                    "article_summary":"",
                    "pub_date":pub_date,
                    "publish_time":publish_time,
                    "harvest_time":harvest_time,
                    "temp_link":temp_link,
                    "publish_source": publish_source,
                    "programme":'null',
                    "feed_class":"News",
                    "publishing_platform":"",
                    "klout_score":"",
                    "journalist":journalist,
                    "headline":headline,
                    "content":content,
                    "source_headline":source_headline,
                    "source_content":source_content,
                    "language":language,
                    "presence":'null',
                    "clip_type":'null',
                    "prog_slot":'null',
                    "op_ed":'0',
                    "location_mention":'',
                    "source_link":source_link,
                    "author_contact":'',
                    "author_emailid":'',
                    "author_url":'',
                    "city":'',
                    "state":'',
                    "country":country,
                    "source":publish_source,
                    "foot_fall":'',
                    "created_on":created_on,
                    "active":'1',
                    'crawl_flag':2,
                    "images_path":images_path,
                    "html_content":html_content,
                    "pdf_url": pdf_url,
                    "pdf_name": pdf_name,
                    "pdf_path":pdf_path
                }

    cl_data.insert_one(clientdata)  
    no_of_data += 1
logger.info('Iteration complete\n')   
logger.info(f'Number of data: {no_of_data}\n')
logger.info(f'Duplicate data: {duplicate_data}\n')
logger.info(f'Unable to fetch article url: {unable_to_fetch_article_url}\n')
logger.info(f'Skipped due to headline: {skipped_due_to_headline}\n')
logger.info(f'Unable to download pdf: {unable_to_download_pdf}\n')
logger.info(f'Skipped due to content: {skipped_due_to_content}\n')
logger.info(f'Skipped due to date: {skipped_due_to_date}\n')
logger.info(f'country: {country}\n')
logger.info(f'language: {language}\n')
logger.info(f'Processing finished in {time.time() - start_time} seconds.\n') 

https://www.spf.org/en/about/annual-report/2019.html
CONTENTS
Pg. 2    History of the Sasakawa Peace Foundation
Pg. 4    Conversation between Honorary Chairman Sasakawa and President Sunami
The role of the Sasakawa Peace Foundation post-COVID-19
Pg. 6    Mission Statement and Five Priority GoalsFY 2019 Program Report 
Pg. 7    Supporting stability in the international community and strengthening Japan–U.S. relations 
Pg. 8    Japan–U.S. Program 
Pg. 10  International Peace and Security Department
Pg. 12  Pacific Island Nations Program 
Pg. 14    Expanding Japan’s presence in Asia
Pg. 15  Asia Peace Initiatives Department 
Pg. 18  Gender Investment and Innovation Department
Pg. 20  Sasakawa Japan–China Friendship Fund 
Pg. 23  Enhancing understanding of and  relationships with Islamic countries
Pg. 24  Middle East and Islam Program Department 
Pg. 27  Establishing ocean governance
Pg. 28  The Ocean Policy Research Institute 
Pg. 32  Financial Position 
Pg. 34  Foundation Profile 
Pg. 34

The Sasakawa Peace Foundation and Conciliation Resources, a UK-based peace NGO, conducted an online public survey in December 2019 to better understand Japanese people's attitudes and perceptions about peacebuilding. A total of 2,000 people responded to the survey, which included questions measuring their understanding of peacebuilding and support for peace activities, as well as questions about how government or civil society actors engage in dialogue with armed groups to resolve conflicts. The questions were based on the survey that previously was conducted in the United Kingdom, Germany, and the United States, allowing for comparisons with other countries. This survey provides new insights into the Japanese people's understanding of peacebuilding and their perceptions of peace efforts.
https://www.spf.org/en/gender/publications/20200915.html
◆About the Report◆
Often referred to as Asia’s last “frontier market”, Myanmar is a country experiencing rapid socio-economic change as a resul

◆About the Report◆

The Emergence of Angel Investment Networks in Southeast Asia is the first research of its kind focusing on Angel Investment Networks (AINs) in Southeast Asia, with a specific focus on Gender Lens Investing practices.

As first report of a three part series, A Good Practice Guide to Effective Angel Investing focuses on the development and current landscape of Angel Investment Networks in Southeast Asia and provides an overview of the functions, taxonomy, and challenges facing Angel Networks in the region. Based on interviews with expert practitioners, this report also provides advice for managers and members of Angel Investment Networks to explore mechanisms to achieve financial sustainability, implement operational improvements, and pursue collaborations with other ecosystem players including entrepreneurs, government, intermediaries, and other investors.

The second report, A Spotlight on Gender Lens Investing (GLI), examines how GLI is being applied by Angel Inves

The Sasakawa Peace Foundation established the Working Group on New Initiatives for Nuclear Energy and Nuclear Non-Proliferation in 2018 to discuss the role of Japan in the field of the civilian use of nuclear energy and global nuclear disarmament. The working group has published two policy recommendations, “International Management of Plutonium: Aiming for Reduction in Plutonium Stocks and Adoption of New International Norms” (May 2019, click here) and “Proposals to the Japanese Government Concerning Denuclearization of North Korea: With a View to Reduction of Nuclear Threat and Establishment of New Security Framework in Northeast Asia” (February 2020, click here).
Keeping in mind the fact that 2020 marks the 75th anniversary of the bombings on Hiroshima and Nagasaki as well as the 50th anniversary of the Treaty on the Non-Proliferation of Nuclear Weapons (NPT)’s entry into force, the working group produced these policy recommendations to outline the contributions Japan can make toward